# Arabic 100k Reviews (Part 1): Preprocessing

## Overview

This notebook is the first part of a comprehensive end-to-end **Natural Language Processing (NLP) pipeline** for Arabic text classification. In this part, we focus on the foundational steps that transform raw Arabic text data into a clean, analyzed dataset ready for machine learning.

## Learning Objectives

By the end of this lab, you will be able to:

1. **Acquire and extract text data** from external sources (Kaggle datasets)
2. **Apply Arabic-specific preprocessing techniques** including:
   - Text cleaning (removing URLs, handles, hashtags)
   - Normalization (removing Arabic elongations and diacritics)
   - Tokenization (breaking text into tokens)
   - Stemming (reducing words to root forms)
   - Stop word removal
3. **Understand the importance of preprocessing order** and why it matters
4. **Perform Exploratory Data Analysis (EDA)** on text data to understand:
   - Class distribution
   - Text length distributions
   - Language detection
   - Vocabulary analysis
   - Word frequency patterns by class
5. **Recognize preprocessing trade-offs** (speed vs. accuracy, information loss vs. efficiency)
6. **Use appropriate tools** for Arabic NLP (PyArabic, NLTK, Qalsadi, CAMeL Tools)

An **NLP pipeline** (as introduced in Module 1) is a systematic sequence of steps that transforms raw text into a format suitable for machine learning models.

**Later, in Part 2:** the goal is to build a model that can classify Arabic reviews into **Positive** or **Negative** sentiment.

---

> In short, we learn: What to do, how to do it, why do it, and where (order).

## Outline

1. **Setup and Imports** - Installing dependencies and importing libraries
2. **Data Acquisition** - Downloading and extracting the Arabic 100k Reviews dataset from Kaggle
3. **Data Loading** - Loading the dataset into a pandas DataFrame
4. **Exploratory Data Analysis (EDA)**:
   - Class distribution analysis
   - Text length distributions
   - Language detection
   - Vocabulary analysis
   - Word frequency patterns by class
5. **Preprocessing Pipeline**:
   - Text cleaning (URLs, handles, hashtags)
   - Normalization (Arabic elongations, diacritics)
   - Tokenization
   - Stemming/Lemmatization
   - Stop word removal
6. **Preprocessing Order** - Understanding why order matters
7. **Preprocessed Data** - Saving cleaned data for Part 2 (classification)

## References

- [NLP Pipeline, Ali Alameer | GitHub](https://github.com/Ali-Alameer/NLP/blob/main/week2_pipeline_part1.ipynb)
- [NLP_Getting_started(Preprocessing), Ali H. El-Kassas | Kaggle](https://www.kaggle.com/code/ali01lulu/03-nlp-getting-started-preprocessing/notebook)

## Glossary of Terms

**Corpus** (plural: corpora): A collection of text documents used for linguistic analysis or training NLP models. In this lab, our corpus is the Arabic reviews dataset.

**Tokenization**: The process of breaking text into smaller units called tokens (words, subwords, or characters). As you learned in Module 1, tokenization is a fundamental step in the NLP pipeline.

**Stemming**: A rule-based technique that reduces words to their root or stem form by removing prefixes and suffixes. For example, "running", "runs", "ran" → "run". Faster but less accurate than lemmatization.

**Lemmatization**: A dictionary-based technique that converts words to their base form (lemma) using linguistic knowledge. More accurate than stemming but slower. For example, "better" → "good" (stemming might not catch this).

**Stop Words**: Common words that appear frequently but carry little semantic meaning (e.g., "the", "is", "at" in English; "في", "من", "إلى" in Arabic). Removing them can reduce vocabulary size and computational overhead.

**Normalization**: The process of converting text to a standard form. This includes case folding (lowercase), removing diacritics (in Arabic), and handling elongations.

**Vocabulary**: The set of unique tokens (words) in a corpus. Building a vocabulary is essential for converting text to numerical representations.

**Data Leakage**: When information from the test set inadvertently influences the training process. We prevent this by splitting data before preprocessing.

**Exploratory Data Analysis (EDA)**: The process of analyzing datasets to summarize their main characteristics, often using visual methods. In NLP, this includes examining class distributions, text lengths, and word frequencies.

**Preprocessing Pipeline**: A sequence of preprocessing steps applied in a specific order. The order matters because later steps depend on earlier ones.

---


## Setup and Imports

We begin by installing necessary packages and importing all required libraries. Grouping imports at the top makes dependencies clear and follows Python best practices.


### Import Libraries

We group imports by category:

1. **Standard Library**: Python built-in modules
2. **Data Science**: NumPy, Pandas for data manipulation
3. **Machine Learning**: Scikit-learn for data splitting and preprocessing
4. **Visualization**: For EDA plots
5. **NLP Libraries**: Arabic-specific and general NLP tools
   - [**PyArabic**](https://github.com/linuxscout/pyarabic): Arabic text manipulation (removing diacritics, elongations)
   - [**tnkeeh (تنقيح)**](https://github.com/ARBML/tnkeeh) is an Arabic preprocessing library based on regex.
   - [**nltk**](https://www.nltk.org/): Natural Language Toolkit (stop words, stemmers)
   - [**qalsadi**](https://github.com/linuxscout/qalsadi): Arabic lemmatizer
   - [**camel-tools**](https://github.com/CAMeL-Lab/camel_tools): CAMeL Tools for advanced Arabic NLP (morphological tokenization)
   - [**ekphrasis**](https://github.com/cbaziotis/ekphrasis): Text processing tool for social media (tokenization, word normalization, word segmentation, spell correction)

In [ ]:
#
# Refer to the README.md for lab setup instructions
#

In [ ]:
# Standard library imports
import random
import zipfile
from collections import Counter

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from spellchecker import SpellChecker # https://pyspellchecker.readthedocs.io/en/latest/

# Machine Learning
from sklearn.model_selection import train_test_split

# Arabic NLP libraries
import pyarabic.araby as araby

print("Libraries imported successfully!")

In [ ]:
# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
nltk.download('stopwords', quiet=True)

# Data acquisition

The data is readily available on Kaggle, thanks to [Abed Khooli](https://www.kaggle.com/datasets/abedkhooli/arabic-100k-reviews) as a `.tsv` file:

The dataset combines reviews from hotels, books, movies, products and a few airlines. It has three classes (Mixed, Negative and Positive). Most were mapped from reviewers' ratings with 3 being mixed, above 3 positive and below 3 negative. Each row has a label and text separated by a tab (tsv). Text (reviews) were cleaned by removing Arabic diacritics and non-Arabic characters. The dataset has no duplicate reviews.

for more information, see [Dataset Card](https://www.kaggle.com/datasets/abedkhooli/arabic-100k-reviews).

1. Download the dataset from Kaggle: `abedkhooli/arabic-100k-reviews`

In [ ]:
!kaggle datasets download -d abedkhooli/arabic-100k-reviews

2. Unzip the file:

In [ ]:
import zipfile

with zipfile.ZipFile('arabic-100k-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall()

## Data extraction

The second step in the NLP pipeline is extracting the text from its native form (such as pdf, image, html, or csv).

1. Load the data into a pandas DataFrame, knowing the separator is a tab (`\t`)

In [ ]:
df_raw = pd.read_csv('ar_reviews_100k.tsv', delimiter='\t')
df_raw

2. Sample 10k examples only to speed things up
   1. Note that we need to stratify the data to keep the same class distribution in the train and test sets.

In [ ]:
# We will use only a subset of the data to speed things up
df_raw, _ = train_test_split(
    df_raw, train_size=10_000, random_state=SEED,
    stratify=df_raw['label'], # keep the same class distribution
)
df_raw = df_raw.reset_index(drop=True)

- If we had our text in, say, HTML or JSON, we would need to use an **HTML parser** or **JSON parser** to extract the relevant text from it.
- Extraction was straight-forward in our case.

## Explore the data

Let's show some text examples.

Since we are dealing with **long texts**, we would need to increase the `max_colwidth` to show the full text in each cell:

In [ ]:
# Show more of the text in each column
pd.set_option('display.max_colwidth', 100)

Here we are using the `display` function to show the dataframe in a nice format. It is what happens when you put the dataframe in a cell by itself:

In [ ]:
from IPython.display import display

# Better than print: display shows the dataframe in a nice format
# This is effectively what happens when you put the dataframe in a cell by itself
print('first 5 rows:')
display(df_raw.head())

print('sample of 5 rows:')
display(df_raw.sample(5))

print('last 5 rows:')
display(df_raw.tail())

Observation: Some words include repeated words like in the following sentence
```
مسلسل قديم ...... ملل .. زهق .. حشو كتيييييييييييييير. ليه صفحة ؟؟ ... في حين ان الحد وتة دي ممك...
```

Observation: some reviews are in Modern Standard Arabic (MSA) and others are in colloquial dialect (اللهجة العامية).

Question: what is the distribution of the data?

In [ ]:
df_raw['label'].value_counts()

Observation: equal distribution for all labels.

# Text cleaning & pre-processing

### Preprocessing depends on multiple factors:

**Language**

- English vs. Arabic
- Different stemmer, lemmatizer, punctuation, digits, tashkeel, ...etc.

**Data Source**
- Tweets: you might need lots of spelling correction, and dealing with edge cases (slang, jargon, emojis, ..etc.)
- Research papers: already proof-read, standard English, better grammer, ..etc.

**Task**
- Keyword-based search: you'd do search with and without stopword removal and combine results to give final answer.
- Translation: everyword is important, can't remove stop-words because word-order and context matters.

Remember, no matter how good your preprocessing is, **"garbage in, garbage out"**. That is, flawed, biased or poor quality ("garbage") input (training data) produces an output (model) of similar ("garbage") quality.

**Order of preprocessing operations does matter**

Example: if you clean by removing punctuations (`@`) you can't remove mentions or handles (`@blabla`) or URLs. You need first to remove handles then remove punctuations.

Also, in English: words like `can't` has an apostrophe, which would be removed. Better: expand contractions: `can't -> cannot`.

### Cleaning Text

- Remove or replace non-useful text (**depends on your end-goal**):
    - extra whitespace: `" \r\n\t"`
    - numbers: `"There are 1250 people" -> "There are [NUM] people"`
    - handles: `"@xxx" -> [HANDLE]`
    - links: `"www.bla.com" [LINK]`
    - flying punctuations (which doesn't add any meaning)
- Outliers
    - Unusually long or short texts

Again, what to keep/remove depends if that is predictive of the task you are trying to achieve: e.g., keyword search vs text classification.

Question: how long are these reviews?

In [ ]:
df_raw['num_chars'] = df_raw['text'].str.len()
df_raw['num_chars'].plot.hist(bins=30, title='Number of characters in each review')

Let's look at the number of words in each review:

In [ ]:
# Let's check if this is the right method chain to extract the number of words
df_raw['text'].str.split().str.len()

In [ ]:
# Let's see if loc[2] is actually 4 words only
df_raw['text'].str.split().loc[2]

In [ ]:
df_raw['num_words'] = df_raw['text'].str.split().str.len()
df_raw['num_words'].plot.hist(bins=30, title='Number of words in each review')

In [ ]:
df_raw[['num_chars', 'num_words']].describe().T

Observation: exponential distribution with most texts being less than 200 words.

Now we shall remove very long / very short texts:

> Note: we call it `df_normal` from now on to indicate the removal of outliers.

In [ ]:
# Keep only the texts below the 90th percentile of the text lengths
df_normal = df_raw[df_raw['num_chars'] <= df_raw['num_chars'].quantile(0.9)]

In [ ]:
df_normal['num_words'].plot.hist(bins=30, title='Number of words in each review')

In [ ]:
df_normal[['num_chars', 'num_words']].describe().T

**Regular Expressions**'s key operations are:
- Match: Determine if a string matches a pattern.
- Find: Locate occurrences of a pattern within a string.
- Replace: Modify a string based on a pattern.
- Split: Divide a string into substrings using a pattern as a delimiter.

See the official [Regular Expression HOWTO](https://docs.python.org/3/howto/regex.html#regex-howto) for a user-friendly introductory tutorial on Regexes.

💡 **Tip**: when writing regexes, we recommended you always use an interactive realtime regex tester like: https://regex101.com/

In [ ]:
import regex as re

# Twitter handles
user_pattern = r"@\w+"
hash_pattern = r"#\w+"

# start with https or start with www or end with .sa
url_pattern = r"https?\S+|www.\S+|\S+\.sa|\S+\.com"

def clean_text(x):
    x = re.sub(user_pattern, '', x)
    x = re.sub(hash_pattern, '', x)
    x = re.sub(url_pattern, '', x)
    return x

In [ ]:
df_normal['text_clean'] = df_normal['text'].map(clean_text)

After any deletion (cleaning), we might end up with empty texts. So let's make sure to remove that:

In [ ]:

df_normal['text_clean'] = df_normal['text_clean'].str.strip()
df_normal = df_normal[df_normal['text_clean'].str.len() > 0]

In [ ]:
# For reproducibility we want to check the same samples
# after any operation on text
sample_ids = df_normal.loc[45:55].index
sample_ids

In [ ]:
df_normal[['text', 'text_clean']].loc[sample_ids]

### Normalizing Words

#### Text normalization techniques

- Lowercase -> `"Car" -> "car"`
- Longataion (التطويل):
    - "الــســلام عـــــــليكم"
- Extra characters denoting extra meaning (add `"!"` to it):
    - `"السلااااااام عليكم" -> "السلام! عليكم"`
    - `"loooooove" -> "love!"`
- Expand contractions (بسط الرموز):
    - Static:
        - `Don't -> Do not`
        - `4you -> for you` (Chat Slang)
    - Contextual: `I’d like to know how I’d done that!`
        - `I'd -> I would`
        - `I'd -> I had`
        - Checkout [pycontractions](https://pypi.org/project/pycontractions/)
- Convert Arabizi and Transliterated text into Arabic:
    - `6areeq / tareek -> طريق`
    - `6a3am / ta’am -> طعام`
    - `Fondoq / fondok -> فندق `
- Slice munged words
    - `heshotwhointhewhatnow -> ['he', 'shot', 'who', 'in', 'the', 'what', 'now']`
    - `thequickbrownfoxjumpsoverthelazydog -> ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']`
    - Checkout: [wordninja](https://github.com/keredson/wordninja)

[**PyArabic**](https://github.com/linuxscout/pyarabic) provides basic functions to manipulate Arabic letters and text, like detecting Arabic letters, Arabic letters groups and characteristics, remove diacritics etc.

In [ ]:
def normalize_text(x):
    # remove punctuations
    x = re.sub(r'[^\w\s]', '', x)

    # remove extra whitespace
    x = re.sub(r'\s+', ' ', x)

    # remove elongations
    x = araby.strip_tatweel(x)
    
    return x

df_normal['text_clean'] = df_normal['text'].apply(normalize_text)
df_normal[['text', 'text_clean']].loc[sample_ids]

Other libraries like [**tnkeeh (تنقيح)**](https://github.com/ARBML/tnkeeh) do exist for Arabic preprocessing of common examples.

#### Correct Typos

[**ekphrasis**](https://github.com/cbaziotis/ekphrasis): 

> Ekphrasis is a text processing tool, geared towards text from social networks, such as Twitter or Facebook. Ekphrasis performs tokenization, word normalization, **word segmentation** (for splitting hashtags) and **spell correction**, using word statistics from 2 big corpora (english Wikipedia, twitter - 330mil english tweets).

```python
from ekphrasis.classes.segmenter import Segmenter

# segmenter using the word statistics from english Wikipedia
seg_eng = Segmenter(corpus="english") 

# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")

words = ["exponentialbackoff", "gamedev", "retrogaming", "thewatercooler", "panpsychism"]
for w in words:
    print(w)
    print("(eng):", seg_eng.segment(w))
    print("(tw):", seg_tw.segment(w))
    print()
```

Output:

```
exponentialbackoff
(eng): exponential backoff
(tw): exponential back off

gamedev
(eng): gamedev
(tw): game dev

retrogaming
(eng): retrogaming
(tw): retro gaming

thewatercooler
(eng): the water cooler
(tw): the watercooler

panpsychism
(eng): panpsychism
(tw): pan psych is m
```

The Spell Corrector is based on [Peter Norvig's spell-corrector](http://norvig.com/spell-correct.html). Just like the segmentation algorithm, we utilize word statistics in order to find the most probable candidate. Besides the provided statistics, you can use your own.

```python
from ekphrasis.classes.spellcorrect import SpellCorrector
sp = SpellCorrector(corpus="english") 
print(sp.correct("korrect"))
```

Output:

```
> correct
```

Let's apply *Spell Correction*:

```python
spell_checker = SpellChecker(language='ar')

def correct_spelling(x):
    misspelled = spell_checker.unknown(x.split())
    for word in misspelled:
        corrected = spell_checker.correction(word)
        if corrected:
            x = x.replace(word, corrected)
    return x
```

**Note**: it actually introduced some errors, rather than fix them.

- "والشعور" -> "والشعوب"
- "لم استمع" -> "لم استسغ"


#### Stemming and Lemmatization

Both simplify words..

**Stemming** (تجريد), the **S**lasher, is a **S**imple, **S**peedy, **Statistical algorithm** that may result in under-stemming and over-stemming.

- Result: Often produces "stems" that aren't real words (e.g., "university" $\rightarrow$ "univers").
- Performance: So fast, but inaccurate.

**Lemmatization** (تصريف), the **L**inguist, is a **L**ookup in **L**ixicon (**Dictionary**) based on **L**anguage rules (**Grammar**).

- Result: Always produces a "lemma," which is a valid dictionary word (e.g., "better" $\rightarrow$ "good").
- Performance: Laid back, but more accurate.

Comparing Qalasadi Lemmatizer vs ISRI Stemmer:

In [ ]:
from nltk.stem import ISRIStemmer
import qalsadi.lemmatizer

# ISRI: Arabic Stemming without a root dictionary
stemmer = ISRIStemmer()
lemmatizer = qalsadi.lemmatizer.Lemmatizer()

words = ["الوقت", "يلعبون", "التلامذة", "سيأكلونها"]

df_stem = pd.DataFrame(words, columns=['word'])
df_stem['stem'] = df_stem['word'].apply(lambda x: stemmer.stem(x))
df_stem['lemma'] = df_stem['word'].apply(lambda x: lemmatizer.lemmatize(x))
df_stem

Giving the lemmatizer context produces a better result:

In [ ]:
text = "يلعب التلامذة في الوقت الذي سيأكلونها فيه"
lemmatizer.lemmatize_text(text)

Let's tokenize the text:

- We will choose to stem the words (takes about 12 seconds), because we don't have much time
- Otherwise, we would have gone with lemmatization because it is more accurate: `use_lemmatizer=True`

In [ ]:
def tokenization(x, use_lemmatizer=False):
    if use_lemmatizer:
        # lemmatize_text is used to be contextual (e.g., verb vs noun)
        # it returns a list of words (tokens)
        x = lemmatizer.lemmatize_text(x)
    else:
        x = araby.strip_tashkeel(x)
        x = araby.tokenize(x)
        x = [stemmer.stem(w) for w in x] # stemming individual words
    return x

df_normal['stemmed_tokens'] = df_normal['text_clean'].apply(lambda x: tokenization(x, use_lemmatizer=False))

In [ ]:
# To demonstrate lemmatizer, we only apply to the first 2 rows
pd.DataFrame({
    'text': df_normal.loc[:2, 'text'],
    'lemma_tokens': df_normal.loc[:2, 'text_clean'].apply(lambda x: tokenization(x, use_lemmatizer=True)),
})

After cleaning, and tokenizing, now we can do stopwords removal.

Note, keeping words should give us more context and hence more information for a more accurate text classification. However, **we choose to remove them to reduce computation overhead on the model**. Less features mean reduced dimensionality and hence, less computation.

#### Stop words

[**Stop words**](https://en.wikipedia.org/wiki/Stop_word) are the words in a stop list (or stoplist or negative dictionary) which are filtered out (i.e. stopped) before or after processing of natural language data (text) because they are deemed insignificant.

There is no single universal list of stop words used by all NLP tools, nor any agreed upon rules for identifying stop words, and indeed not all tools even use such a list. Therefore, any group of words can be chosen as the stop words for a given purpose. The "general trend in [information retrieval] systems over time has been from standard use of quite large stop lists (200–300 terms) to very small stop lists (7–12 terms) to no stop list whatsoever".

In [ ]:
stopwords_ar = nltk.corpus.stopwords.words('arabic')

print(len(stopwords_ar)) # how many stop words?
print(stopwords_ar[:10]) # first 10 stop words

In [ ]:
stopwords_ar.append('جميع') # add a custom stopword
stopwords_ar.append('السلام عليكم ورحمة الله وبركاته') # add a custom phrase

In [ ]:
print(len(stopwords_ar)) # should be more now
print(stopwords_ar[-5:]) # last 5 stop words now

In [ ]:
def remove_stopwords(xs):
    return [w for w in xs if w not in stopwords_ar]

df_normal['stemmed_tokens'] = df_normal['stemmed_tokens'].apply(remove_stopwords)

In [ ]:
df_normal.loc[sample_ids, ['text_clean', 'stemmed_tokens']]

### Morphological Tokenization  (Optional)

Morphological tokenization whereby Arabic words are split into component prefixes, stems, and suffixes.

- Input Tokens: `['فتنفست', 'الصعداء']`
- Output: split morphologically and apply diacritics: `['فَ+', 'تَنَفَّسْتُ', 'ال+', 'صُعَداءَ']`

[CAMeL Tools](https://github.com/CAMeL-Lab/camel_tools) is suite of Arabic natural language processing tools developed by the CAMeL Lab at New York University Abu Dhabi.

See the [CAMeL Tools Guided Tour](https://colab.research.google.com/drive/1Y3qCbD6Gw1KEw-lixQx1rI6WlyWnrnDS?usp=sharing#scrollTo=KmeKhpPVsI69); Morphological Tokenization.

#### Install CAMeL Tools Data

CAMeL Tools requires additional data files for morphological analysis. We install the "light" version for this lab.

In [ ]:
# %pip install camel-tools

In [ ]:
# install just the datasets for morphology and MLE disambiguation only
# !camel_data -i light

In [ ]:
from camel_tools.tokenizers.word import simple_word_tokenize
from camel_tools.disambig.mle import MLEDisambiguator
from camel_tools.tokenizers.morphological import MorphologicalTokenizer

# The tokenizer expects pre-tokenized text
sentence = simple_word_tokenize('فتنفست الصعداء')
print(sentence)

# Load a pretrained disambiguator to use with a tokenizer
mle = MLEDisambiguator.pretrained('calima-msa-r13')

# We can output diacritized tokens by setting `diac=True`
tokenizer = MorphologicalTokenizer(mle, scheme='d3tok',
                                   split=True, # split affixes
                                   diac=True,  # diacritize
                                   )
tokens = tokenizer.tokenize(sentence)
print(tokens)

In [ ]:
def tokenize_morph(x):
    x = simple_word_tokenize(x)
    return tokenizer.tokenize(x)

# Due to processing time, we only demonstrate it
print(df_normal.loc[:5, 'text_clean'].apply(tokenize_morph).apply(lambda xs: ' '.join(xs)).str.cat(sep='\n'))

**Note**: to make the tutorial light and quick, we will not use **morphological tokenization** (it is computationally heavy). However, we think it is much more accurate to do such tokenization than simple *lemmatizers* or *stemmers*.

### Save the cleaned data

In [ ]:
df_normal

In [ ]:
df_normal.to_csv('ar_reviews_100k_cleaned.tsv', sep='\t', index=False)

## Key Takeaways

- **Data acquisition** from external sources (like Kaggle) requires proper authentication and data extraction techniques.

- **Exploratory Data Analysis (EDA)** is essential before preprocessing to understand:
  - Class distribution (balanced vs. imbalanced datasets)
  - Text length distributions (identifying outliers)
  - Language detection (ensuring expected language)
  - Vocabulary characteristics (size, frequency patterns)
  - Class-specific word patterns

- **Data splitting** must occur before preprocessing to prevent data leakage from test set to training set.

- **Preprocessing order matters critically**:
  1. Extract structured information (URLs, handles, hashtags)
  2. Normalize text (remove elongations, diacritics)
  3. Tokenize
  4. Stem/Lemmatize
  5. Remove stop words last
 
- **Preprocessing trade-offs** exist between:
  - Speed vs. accuracy (stemming vs. lemmatization)
  - Simple models needs simple features and vice-versa (stop words removal)
  - Vocabulary size vs. computational cost

- Understanding your data through EDA guides preprocessing decisions and helps build effective NLP pipelines for Arabic text classification.